# Load dataset

In [34]:
from glob import glob
import os
import json 

path = './data/extended_esc_13b.json'

with open(path, 'r') as f:
    data = json.load(f)
        

print(len(data))

9354


In [36]:
lens = [len(x['dialog']) for x in data]

print(min(lens))

min_dialog_turns = min(lens)
    

5


# Process conversations and add an id to them

In [39]:
from uuid import uuid4

conv_ids = {}

for ex in data:
    conv_str = '_'.join(ex['dialog'][:min_dialog_turns])
    if conv_str not in conv_ids:
        conv_ids[conv_str] = str(uuid4())
        
    ex['id'] = conv_ids[conv_str]

In [40]:
print("total conversations:", len(conv_ids))

total conversations: 1297


# split across conversations for training and test

In [42]:
import numpy as np

all_ids = list(conv_ids.values())
np.random.shuffle(all_ids)

val_size = 50
test_size = 100

test_ids = all_ids[:test_size]
val_ids = all_ids[test_size:test_size+val_size]
train_ids = all_ids[test_size+val_size:]

train = [d for d in data if d['id'] in train_ids]
val = [d for d in data if d['id'] in val_ids]
test = [d for d in data if d['id'] in test_ids]

print("training size: ", len(train))
print("validation size: ", len(val))
print("test size: ", len(test))

training size:  8278
validation size:  359
test size:  717


In [43]:
import json

def write_jsonl(path, data):
    with open(path, 'w') as f:
        for ex in data:
            f.write(json.dumps(ex) + "\n")
            
            
write_jsonl('data/train.json', train)
write_jsonl('data/dev.json', val)
write_jsonl('data/test.json', test)